# CIFAR10 Classifier - Softmax loss

### Import packages

In [1]:
import numpy as np

import torch
import torchvision
import torchvision.transforms as transforms

import matplotlib.pyplot as plt

### Load and normalize CIFAR10

In [2]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 4

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


### NEURAL NETWORK WITH SOFTMAX FUNCTION

In [3]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

### Loss function and optimizer

In [ ]:
import torch.optim as optim

optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [4]:
def my_cross_entropy(x,y):
    log_prob = -1.0 * F.log_softmax(x, 1)
    loss = log_prob.gather(1, y.unsqueeze(1))
    loss = loss.mean()
    return loss

### Training network

In [5]:
import torch.optim as optim


epochs = 8
for e in range(epochs):
  running_loss = 0
  for i, data in enumerate(trainloader, 0):
    # get the inputs; data is a list of [inputs, labels]
    inputs, labels = data

    # zero the parameter gradients
    optimizer.zero_grad()

    # forward + backward + optimize
    outputs = net(inputs)
    loss = my_cross_entropy(outputs, labels)
    loss.backward()
    optimizer.step()

    # print statistics
    running_loss += loss.item()
    if i % 1000 == 999:    # print every 2000 mini-batches
        print(f'[{e + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
        running_loss = 0.0

print('Finished Training')

/mnt/data/klaszewski/miniconda3/envs/nn_proj/lib/python3.9/site-packages/torch/autograd/__init__.py:154: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 101: invalid device ordinal (Triggered internally at  /opt/conda/conda-bld/pytorch_1640811803361/work/c10/cuda/CUDAFunctions.cpp:112.)
  Variable._execution_engine.run_backward(


[1,  1000] loss: 1.146
[1,  2000] loss: 1.055
[1,  3000] loss: 0.963
[1,  4000] loss: 0.888
[1,  5000] loss: 0.851
[1,  6000] loss: 0.833
[1,  7000] loss: 0.802
[1,  8000] loss: 0.807
[1,  9000] loss: 0.783
[1, 10000] loss: 0.763
[1, 11000] loss: 0.766
[1, 12000] loss: 0.755
[2,  1000] loss: 0.721
[2,  2000] loss: 0.714
[2,  3000] loss: 0.713
[2,  4000] loss: 0.712
[2,  5000] loss: 0.686
[2,  6000] loss: 0.689
[2,  7000] loss: 0.675
[2,  8000] loss: 0.660
[2,  9000] loss: 0.659
[2, 10000] loss: 0.653
[2, 11000] loss: 0.653
[2, 12000] loss: 0.648
[3,  1000] loss: 0.623
[3,  2000] loss: 0.628
[3,  3000] loss: 0.603
[3,  4000] loss: 0.619
[3,  5000] loss: 0.604
[3,  6000] loss: 0.603
[3,  7000] loss: 0.597
[3,  8000] loss: 0.596
[3,  9000] loss: 0.613
[3, 10000] loss: 0.606
[3, 11000] loss: 0.604
[3, 12000] loss: 0.595
[4,  1000] loss: 0.560
[4,  2000] loss: 0.548
[4,  3000] loss: 0.559
[4,  4000] loss: 0.560
[4,  5000] loss: 0.569
[4,  6000] loss: 0.563
[4,  7000] loss: 0.562
[4,  8000] 

### Test network on dataset

In [6]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in testloader:
        images, labels = data
        # calculate outputs by running images through the network
        outputs = net(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')

Accuracy of the network on the 10000 test images: 62 %


### Predictions for each class

In [7]:
# prepare to count predictions for each class
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

# again no gradients needed
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predictions = torch.max(outputs, 1)
        # collect the correct predictions for each class
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1


# print accuracy for each class
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')


Accuracy for class: plane is 56.8 %
Accuracy for class: car   is 68.9 %
Accuracy for class: bird  is 49.2 %
Accuracy for class: cat   is 55.3 %
Accuracy for class: deer  is 46.8 %
Accuracy for class: dog   is 53.0 %
Accuracy for class: frog  is 73.4 %
Accuracy for class: horse is 66.7 %
Accuracy for class: ship  is 80.1 %
Accuracy for class: truck is 76.8 %
